📖 Análise e Geração de Relatórios de E-commerce: Explicação Detalhada

Este Notebook Jupyter serve como um guia completo para o projeto de análise de vendas de e-commerce. Aqui, exploraremos cada etapa do código, desde o carregamento dos dados e seu pré-processamento, passando pela interação com um banco de dados SQLite, até a geração de relatórios visuais. O projeto é estruturado com Programação Orientada a Objetos (POO) para modularidade e boa prática de código.

📦 1. Importação de Bibliotecas
Iniciamos importando todas as bibliotecas necessárias para as operações de manipulação de dados, visualização e interação com o banco de dados.

In [ ]:
# Célula de Código 1
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os # Necessário para interagir com o sistema de arquivos, como criar diretórios

⚙️ 2. Configurações Iniciais de Visualização
Definimos algumas configurações para melhorar a estética e o tamanho padrão dos gráficos gerados pelo Matplotlib e Seaborn.

In [ ]:
# Célula de Código 2
sns.set_style("whitegrid") # Define um estilo de fundo com grade branca para os gráficos
plt.rcParams['figure.figsize'] = (10, 6) # Define o tamanho padrão (largura, altura) para todas as figuras

📝 3. Classe DataManager
A classe DataManager é responsável por todas as operações relacionadas ao gerenciamento dos dados. Isso inclui carregar o arquivo CSV, realizar o pré-processamento essencial (como conversão de tipos e criação de novas colunas) e fornecer os dados já prontos para análise ou outras partes do sistema. Ela encapsula a lógica de manipulação do DataFrame.

In [ ]:
# Célula de Código 3
class DataManager:
    """
    Gerencia o carregamento, pré-processamento e acesso aos dados de vendas.
    """
    def __init__(self, csv_file):
        """
        O construtor da classe DataManager.
        Inicializa o DataManager com o caminho para o arquivo CSV e um DataFrame vazio.

        Args:
            csv_file (str): O caminho para o arquivo CSV de vendas.
        """
        self.csv_file = csv_file
        self.df_vendas = pd.DataFrame() # Inicializa um DataFrame vazio para os dados de vendas

    def load_data(self):
        """
        Carrega os dados do CSV para o DataFrame 'df_vendas'.
        Inclui tratamento de erro caso o arquivo não seja encontrado ou haja problemas de leitura.

        Returns:
            bool: True se o carregamento for bem-sucedido, False caso contrário.
        """
        if not os.path.exists(self.csv_file):
            print(f"Erro: O arquivo '{self.csv_file}' não foi encontrado.")
            return False
        try:
            self.df_vendas = pd.read_csv(self.csv_file)
            print(f"Arquivo '{self.csv_file}' carregado com sucesso.")
            return True
        except Exception as e:
            print(f"Erro ao carregar o arquivo CSV: {e}")
            return False

    def preprocess_data(self):
        """
        Realiza o pré-processamento essencial nos dados carregados:
        - Converte a coluna 'Data_Venda' para o tipo datetime, permitindo operações temporais.
        - Calcula 'Valor_Total_Venda' multiplicando 'Preco_Unitario' por 'Quantidade_Vendida'.
        - Extrai 'Mes_Venda' no formato 'AAAA-MM' para agrupamentos mensais.
        """
        if self.df_vendas.empty:
            print("DataFrame vazio. Carregue os dados primeiro para pré-processá-los.")
            return

        # Converte a coluna 'Data_Venda' para o tipo datetime.
        # Isso é crucial para que o Pandas reconheça e manipule as datas corretamente.
        self.df_vendas['Data_Venda'] = pd.to_datetime(self.df_vendas['Data_Venda'])

        # Calcula o valor total de cada transação.
        # É uma nova feature (característica) que agregamos aos dados existentes.
        self.df_vendas['Valor_Total_Venda'] = self.df_vendas['Preco_Unitario'] * self.df_vendas['Quantidade_Vendida']

        # Extrai o mês e o ano da 'Data_Venda' para análises temporais por mês.
        # O formato 'M' cria um objeto Period (AAAA-MM), ideal para agrupamento.
        self.df_vendas['Mes_Venda'] = self.df_vendas['Data_Venda'].dt.to_period('M')
        print("Dados pré-processados com sucesso.")

    def get_df_vendas(self):
        """
        Retorna o DataFrame de vendas processado.
        Outras partes do programa podem acessar os dados limpos e enriquecidos através deste método.

        Returns:
            pd.DataFrame: O DataFrame de vendas com todas as transformações aplicadas.
        """
        return self.df_vendas

    def get_sales_by_category(self):
        """
        Calcula as vendas totais agrupadas por categoria de produto.

        Returns:
            pd.Series: Uma Série Pandas com as categorias como índice e o valor total de vendas.
        """
        # Agrupa o DataFrame pela coluna 'Categoria_Produto' e soma o 'Valor_Total_Venda' para cada grupo.
        # 'sort_values(ascending=False)' ordena do maior para o menor valor.
        return self.df_vendas.groupby('Categoria_Produto')['Valor_Total_Venda'].sum().sort_values(ascending=False)

    def get_sales_by_region(self):
        """
        Calcula as vendas totais agrupadas por região do cliente.

        Returns:
            pd.Series: Uma Série Pandas com as regiões como índice e o valor total de vendas.
        """
        # Agrupa pela coluna 'Regiao_Cliente' e soma o 'Valor_Total_Venda'.
        return self.df_vendas.groupby('Regiao_Cliente')['Valor_Total_Venda'].sum().sort_values(ascending=False)

    def get_sales_by_month(self):
        """
        Calcula as vendas totais agrupadas por mês.

        Returns:
            pd.DataFrame: Um DataFrame com 'Mes_Venda' e 'Valor_Total_Venda', ordenado cronologicamente.
        """
        # Agrupa por 'Mes_Venda', soma 'Valor_Total_Venda', ordena pelo índice (mês) e reseta o índice.
        # O reset_index() transforma o índice 'Mes_Venda' em uma coluna comum para facilitar a plotagem no Seaborn.
        return self.df_vendas.groupby('Mes_Venda')['Valor_Total_Venda'].sum().sort_index(ascending=True).reset_index()

🗄️ 4. Classe DatabaseManager
A classe DatabaseManager é responsável por toda a interação com o banco de dados SQLite. Ela gerencia a conexão, a criação da tabela, a inserção de dados do DataFrame e a recuperação de dados do banco. Isso isola a lógica de banco de dados do restante da aplicação.

In [ ]:
# Célula de Código 4
class DatabaseManager:
    """
    Gerencia a conexão e as operações com o banco de dados SQLite.
    """
    def __init__(self, db_file):
        """
        O construtor da classe DatabaseManager.
        Inicializa o DatabaseManager com o caminho para o arquivo do banco de dados SQLite.

        Args:
            db_file (str): O caminho para o arquivo do banco de dados SQLite (ex: 'ecommerce_data.db').
        """
        self.db_file = db_file
        self.conn = None # Atributo para armazenar a conexão com o banco de dados
        self.cursor = None # Atributo para armazenar o cursor do banco de dados

    def connect(self):
        """
        Estabelece a conexão com o banco de dados SQLite.
        Se o arquivo do banco de dados não existir, ele será criado automaticamente.

        Returns:
            bool: True se a conexão for bem-sucedida, False caso contrário.
        """
        try:
            self.conn = sqlite3.connect(self.db_file)
            self.cursor = self.conn.cursor() # Cria um cursor para executar comandos SQL
            print(f"Conexão com o banco de dados '{self.db_file}' estabelecida com sucesso.")
            return True
        except sqlite3.Error as e:
            print(f"Erro ao conectar ao banco de dados: {e}")
            return False

    def create_table(self):
        """
        Cria a tabela 'vendas' no banco de dados, se ela ainda não existir.
        Define o esquema da tabela com os tipos de dados apropriados para o SQLite.

        Returns:
            bool: True se a tabela for criada/verificada com sucesso, False caso contrário.
        """
        if not self.conn:
            print("Não há conexão com o banco de dados. Conecte-se primeiro para criar a tabela.")
            return False
        try:
            self.cursor.execute('''
                CREATE TABLE IF NOT EXISTS vendas (
                    ID_Venda INTEGER PRIMARY KEY,
                    Data_Venda TEXT,
                    Nome_Produto TEXT,
                    Categoria_Produto TEXT,
                    Preco_Unitario REAL,
                    Quantidade_Vendida INTEGER,
                    Cliente_ID TEXT,
                    Regiao_Cliente TEXT,
                    Valor_Total_Venda REAL,
                    Mes_Venda TEXT -- Coluna adicionada para armazenar o mês como texto (AAAA-MM)
                )
            ''')
            self.conn.commit() # Confirma as alterações (criação da tabela) no banco de dados
            print("Tabela 'vendas' verificada/criada com sucesso.")
            return True
        except sqlite3.Error as e:
            print(f"Erro ao criar tabela: {e}")
            return False

    def insert_data(self, df_data):
        """
        Insere dados de um DataFrame Pandas na tabela 'vendas' do SQLite.
        Utiliza a função to_sql() do Pandas, que é eficiente para essa tarefa.

        Args:
            df_data (pd.DataFrame): O DataFrame contendo os dados a serem inseridos.

        Returns:
            bool: True se a inserção for bem-sucedida, False caso contrário.
        """
        if not self.conn:
            print("Não há conexão com o banco de dados. Conecte-se primeiro para inserir dados.")
            return False
        try:
            # Cria uma cópia do DataFrame e converte 'Mes_Venda' para string.
            # Isso é necessário porque o SQLite não suporta o tipo 'Period' diretamente.
            df_to_insert = df_data.copy()
            df_to_insert['Mes_Venda'] = df_to_insert['Mes_Venda'].astype(str)

            # Usa o método to_sql() do Pandas para inserir o DataFrame na tabela 'vendas'.
            # 'if_exists='replace'' apaga e recria a tabela se ela já existir.
            # 'index=False' evita que o índice do DataFrame seja salvo como uma coluna na tabela.
            df_to_insert.to_sql('vendas', self.conn, if_exists='replace', index=False)
            self.conn.commit() # Confirma a inserção dos dados no banco
            print("Dados do DataFrame inseridos na tabela 'vendas' com sucesso.")
            return True
        except sqlite3.Error as e:
            print(f"Erro ao inserir dados: {e}")
            return False

    def fetch_all_vendas(self):
        """
        Recupera todos os dados da tabela 'vendas' do SQLite e os carrega em um novo DataFrame Pandas.

        Returns:
            pd.DataFrame: Um DataFrame Pandas contendo os dados recuperados do banco de dados,
                          ou um DataFrame vazio em caso de erro ou sem conexão.
        """
        if not self.conn:
            print("Não há conexão com o banco de dados. Conecte-se primeiro para recuperar dados.")
            return pd.DataFrame()
        try:
            # Executa uma consulta SQL para selecionar todas as colunas da tabela 'vendas'.
            # pd.read_sql_query() executa a consulta e retorna os resultados como um DataFrame.
            df_from_db = pd.read_sql_query("SELECT * FROM vendas", self.conn)
            print("Dados recuperados do SQLite com sucesso.")
            # Opcional: converte 'Data_Venda' de volta para datetime, pois foi salva como TEXT no SQLite.
            df_from_db['Data_Venda'] = pd.to_datetime(df_from_db['Data_Venda'])
            return df_from_db
        except sqlite3.Error as e:
            print(f"Erro ao recuperar dados: {e}")
            return pd.DataFrame()

    def close_connection(self):
        """
        Fecha a conexão ativa com o banco de dados SQLite.
        É uma boa prática fechar a conexão para liberar recursos.
        """
        if self.conn:
            self.conn.close()
            print("Conexão com o banco de dados fechada.")

🚀 5. Lógica Principal do Programa (Orquestrador)
Esta seção orquestra as operações, instanciando as classes DataManager e DatabaseManager e chamando seus métodos em sequência para executar o fluxo completo do projeto: carregar dados, processá-los, persistir no banco de dados e gerar visualizações.

In [ ]:
# Célula de Código 5
if __name__ == "__main__":
    # Define os caminhos para o arquivo CSV, o arquivo do banco de dados e o diretório de saída dos gráficos.
    csv_path = 'vendas.csv'
    db_path = 'ecommerce_data.db'
    output_dir = 'relatorios_imagens'

    # Cria o diretório de saída para os gráficos se ele ainda não existir.
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Diretório '{output_dir}' criado para salvar os relatórios.")

    # --- Etapa 1: Gerenciamento e Pré-processamento de Dados ---
    print("\n--- Iniciando Gerenciamento de Dados ---")
    data_manager = DataManager(csv_path) # Instancia o DataManager
    if data_manager.load_data(): # Tenta carregar os dados do CSV
        data_manager.preprocess_data() # Pré-processa os dados (conversão de tipo, cálculo de valor total)
        df_vendas = data_manager.get_df_vendas() # Obtém o DataFrame processado

        # --- Etapa 2: Gerenciamento do Banco de Dados ---
        print("\n--- Iniciando Gerenciamento do Banco de Dados ---")
        db_manager = DatabaseManager(db_path) # Instancia o DatabaseManager
        if db_manager.connect(): # Tenta conectar ao banco de dados
            db_manager.create_table() # Cria a tabela de vendas (se não existir)
            db_manager.insert_data(df_vendas) # Insere os dados processados do DataFrame no banco

            # Opcional: Recupera os dados do banco de dados para verificar se a inserção foi bem-sucedida.
            df_vendas_from_db = db_manager.fetch_all_vendas()
            print("\n--- Primeiras 5 linhas do DataFrame lido do DB (verificação) ---")
            print(df_vendas_from_db.head()) # Exibe as primeiras linhas do DataFrame recuperado
            print("\n--- Informações do DataFrame lido do DB (verificação) ---")
            df_vendas_from_db.info() # Exibe informações sobre os tipos de dados do DataFrame recuperado
            db_manager.close_connection() # Fecha a conexão com o banco de dados

        # --- Etapa 3: Geração e Exibição de Visualizações ---
        print("\n--- Gerando e Salvando Visualizações ---")

        # 3.1. Gráfico de Vendas por Categoria de Produto
        vendas_por_categoria = data_manager.get_sales_by_category() # Obtém os dados agregados
        plt.figure(figsize=(10, 6)) # Cria uma nova figura para o gráfico
        sns.barplot(x=vendas_por_categoria.index, y=vendas_por_categoria.values, palette='viridis') # Cria o gráfico de barras
        plt.title('Vendas Totais por Categoria de Produto') # Define o título do gráfico
        plt.xlabel('Categoria de Produto') # Rótulo do eixo X
        plt.ylabel('Valor Total de Vendas (R$)') # Rótulo do eixo Y
        plt.xticks(rotation=45, ha='right') # Gira os rótulos do eixo X para melhor legibilidade
        plt.tight_layout() # Ajusta o layout para evitar que elementos se sobreponham
        plt.savefig(os.path.join(output_dir, 'vendas_por_categoria.png')) # Salva o gráfico como PNG
        plt.show() # Exibe o gráfico na tela

        # 3.2. Gráfico de Vendas por Região do Cliente
        vendas_por_regiao = data_manager.get_sales_by_region() # Obtém os dados agregados
        plt.figure(figsize=(10, 6)) # Nova figura
        sns.barplot(x=vendas_por_regiao.index, y=vendas_por_regiao.values, palette='magma') # Cria o gráfico de barras
        plt.title('Vendas Totais por Região do Cliente')
        plt.xlabel('Região do Cliente')
        plt.ylabel('Valor Total de Vendas (R$)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'vendas_por_regiao.png')) # Salva o gráfico
        plt.show() # Exibe o gráfico

        # 3.3. Gráfico de Tendência de Vendas por Mês
        vendas_por_mes = data_manager.get_sales_by_month() # Obtém os dados agregados por mês
        plt.figure(figsize=(12, 6)) # Nova figura
        # Para o gráfico de linhas, convertemos 'Mes_Venda' para string para garantir a plotagem correta
        sns.lineplot(x=vendas_por_mes['Mes_Venda'].astype(str), y='Valor_Total_Venda', data=vendas_por_mes, marker='o')
        plt.title('Tendência de Vendas Totais por Mês')
        plt.xlabel('Mês da Venda')
        plt.ylabel('Valor Total de Vendas (R$)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'vendas_por_mes.png')) # Salva o gráfico
        plt.show() # Exibe o gráfico

    else:
        print("Não foi possível prosseguir devido a erros no carregamento ou pré-processamento dos dados.")